# Music play library ETL Pipeline
The following program will create an ETL pipeline that reads content in from csv files in a data folder, creates a single csv with the data that be used and then transfers the most critical data to a Cassandra database for the highest availability of the desired queries that will be used.

## Part I. The following code will pre-process a folder of CSV files that contain music play history into a single csv file that contains the following information:
* artist
* firstName
* gender
* itemInSession
* lastName
* length
* level
* location
* sessionId
* song
* userId

The files will be merged into a single csv file for use further in the program enter the data into a Cassandra Database.

#### Import Python packages 

In [90]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [91]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [92]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [93]:
# Cecking  the number of rows in the result csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Loading the data from the csv file event_datafile_new.csv into the Cassandra database to address the most critical queries that will be run on the data.

1. Querying the data for artist, song title and song's length by sessionId and itemInSession
2. Querying the data for artist, song title (sorted by itemInSession) and user (first and last name) by userid and sessionid
3. Querying the data for user name (first and last) by song title



## Creating a Cassandra instance and connecting to it.

#### Creating a Cluster

In [94]:
# Creating a connection to a Cassandra instance locally
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])

    # Creating a session 
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating Keyspace

In [95]:
# Creating the keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    )
except Expection as e:
    print(e)

#### Setting Keyspace

In [96]:
# Setting the keyspace
try: 
    session.set_keyspace('udacity')
except Exception as e:
    print(e)
    
    


## Creating the desired tables in Cassandra 

Note:  Information about key setup can be found here:  https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra

## Table 1:  play_information 
Columns
* sessionId
* itemInSession
* artist
* song
* length

The composite partition key is (sessionId and itemInSession) 

In [97]:
# Creating the table play_information 
# The composite partition key is (sessionId and itemInSession) 
# This table will be useful for querying particular play sessions.

query = "CREATE TABLE IF NOT EXISTS play_information "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal,\
PRIMARY KEY((sessionId, itemInSession)))"

try:
    session.execute(query)
except Exception as e:
    print(e)


                    

## Table 2:  user_plays
Columns
* sessionId
* itemInSession
* userId
* artist
* song
* firstname
* lastname

The composite partition key is (sessionId and itemInSession) the clustering key is userId.

In [98]:
# Creating the table user_plays which focuses on individual user sessions.
# The The composite partition key is (userId , sessionId), the clustering key is itemInSession.
# This table will be useful for querying particular plays by user.

#### Need to improve description above and fix primary key vs clustering info

query = "CREATE TABLE IF NOT EXISTS user_plays "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, \
firstname text, lastname text, PRIMARY KEY((userId, sessionId),itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

## Table 3:  songs_played
Columns
* userId
* song
* firstname
* lastname

The composite partition key is (userId and song) 

In [99]:
# Creating the table songs_played to store the values for the query
# The composite partition key is (userId and song) 
# This table will be useful for seeing what user played certain songs.

query = "CREATE TABLE IF NOT EXISTS songs_played "
query = query + "(song text,userId int, firstname text, lastname text,\
PRIMARY KEY(song,userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

## Insert into data table 1: play_information

In [100]:

# Inserting content into the play_information tables from the csv file that contains the data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Insert statement that will be used
        query = "INSERT INTO play_information (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Executing inserts
        try: 
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
                                                                        

## Insert into data table 2: user_plays

In [101]:
#  Inserting content into the user_plays from the csv file that contains the data 
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Insert statement that will be used
        query = "INSERT INTO user_plays (userId, sessionId, itemInSession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
       # Execute insert
        try: 
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1],line[4]))
        except Exception as e:
            print(e)

## Insert into data table 3: songs_played

In [102]:
#  Transferring the data from the csv to songs_played table
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Insert statement that will be used
        query = "INSERT INTO songs_played (userId, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Execute Insert
        try: 
            session.execute(query, (int(line[10]),line[9], line[1],line[4]))
        except Exception as e:
            print(e)

## Test Select statements to verify the data is correct

## Table 1 play_information select

In [103]:
# Selecting data from play_inforamtion based on sessionId and ItemInSession information.

query = "select artist, song, length from play_information \
where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Table 2 user_plays select

In [104]:
# Selecting data from user_plays based on userid and sessionid

query = "select artist, song, firstname, lastname from user_plays \
where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Table 3 songs_played select

In [105]:
# Selecting data from songs_played based on song name

query = "select firstname,lastname from songs_played \
where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [106]:

# Dropping the three tables that have been created.

query = "drop table if exists play_information"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists user_plays"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists songs_played"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [107]:
session.shutdown()
cluster.shutdown()